In [6]:
import numpy as np
import pandas as pd

from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

In [ ]:
def plan_lines_deep_to_shallow(
    theta_deg=120,  # 换开角 θ
    alpha_deg=1.5,  # 坡度 α
    eta=0.10,  # 期望重叠率 η  (10 %–20 % 之间取值)
    D_center=110,  # 海域中心深度  (m)
    L_ew_nm=4.0,  # 东-西总跨度 (nmi)
):
    """
    由西(深)向东(潜)逐条布线，返回每条测线的中心横坐标 x 和水深 D。
    x=0 定义在西侧边界，正向指向东侧。
    """
    # 常量与单位换算
    nmi2m = 1852.0
    L_x = L_ew_nm * nmi2m  # 东-西长度 (m)
    theta = np.radians(theta_deg)
    alpha = np.radians(alpha_deg)

    # —— 1. 计算西端(最深)水深 D0 ——
    half_len = L_x / 2  # 西边界到中心距离
    D0 = D_center + half_len * np.tan(alpha)  # 最深处 (x=0) 水深
    # print(f"中心水深 D_center = {D_center:.2f} m")
    # print(f"西端水深 D0 = {D0:.2f} m")

    # —— 2. 第一条测线 ——
    d1 = D0 * np.tan(theta / 2)  # 与边界距离
    x_list = [d1]  # 第 1 条测线横坐标
    D_list = [D0 - d1 * np.tan(alpha)]  # 中心深度函数 D(x)
    W_list = [] # 测线宽度列表 (未使用)

    # —— 3. 迭代生成后续测线 ——
    while True:
        Dk = D_list[-1]  # 当前(第 k 条)水深
        W_left = Dk * np.sin(theta / 2) / np.cos(theta / 2 + alpha)
        W_right = Dk * np.sin(theta / 2) / np.cos(theta / 2 - alpha)
        Wk = W_left + W_right  # 当前测线宽度
        W_list.append(Wk)
        # 间距公式  (题设给出)
        # delta = (
        #     (1 - eta)
        #     * Dk
        #     * (1 + np.cos(theta / 2 + alpha) * (1 / np.cos(theta / 2 - alpha)))
        # ) / (1 - np.tan(alpha))
        delta = Wk*(1-eta)*np.cos(alpha)/(1+np.sin(alpha)*np.sin(theta/2)/np.cos(theta/2+alpha)*(1-eta))
        next_x = x_list[-1] + delta

        # 到达或超出东侧边界即停止
        if next_x >= L_x:
            break

        # 下一测线水深
        D_next = D0 - next_x * np.tan(alpha)

        x_list.append(next_x)
        D_list.append(D_next)

    # —— 4. 汇总为 DataFrame ——
    df = pd.DataFrame(
        {
            "line_no": np.arange(1, len(x_list) + 1),
            "x_from_west(m)": np.round(x_list, 2),
            "depth_center(m)": np.round(D_list, 2),
        }
    )
    return df

In [13]:
df_lines = plan_lines_deep_to_shallow()
print(df_lines)

L_sn_nm = 2.0  # 南-北总跨度 (nmi)
line_num = len(df_lines)
tot = L_sn_nm * line_num  # 总长度 (nmi)
print(f"测线数量 = {line_num}")
print(f"总长度 = {tot:.2f} nmi")
df_lines.to_excel("result3_deep_to_shallow.xlsx", index=False)
print("结果已保存到 result3_deep_to_shallow.xlsx")

    line_no  x_from_west(m)  depth_center(m)
0         1          358.52           197.60
1         2          950.55           182.10
2         3         1496.13           167.82
3         4         1998.90           154.65
4         5         2462.23           142.52
5         6         2889.22           131.34
6         7         3282.70           121.03
7         8         3645.31           111.54
8         9         3979.48           102.79
9        10         4287.43            94.72
10       11         4571.22            87.29
11       12         4832.74            80.44
12       13         5073.75            74.13
13       14         5295.85            68.32
14       15         5500.52            62.96
15       16         5689.14            58.02
16       17         5862.96            53.47
17       18         6023.15            49.27
18       19         6170.76            45.41
19       20         6306.80            41.84
20       21         6432.16            38.56
21       2